# sqlmodel Tutorial

## Cleanup

In [1]:
!rm tmp/tutorial.db

## Tutorial

In [2]:
from typing import Optional, List

from sqlmodel import Field, Relationship, SQLModel, Session
from sqlmodel import create_engine, select

import sqlmodel

In [3]:
class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int]
    team_id: Optional[int] = Field(default=None, foreign_key="team.id")
    team: Optional["Team"] = Relationship(back_populates="heroes")


class Team(SQLModel, table=True):
    """Team."""
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    heroes: List["Hero"] = Relationship(back_populates="team")

In [4]:
Hero(
    name="Batman",
    secret_name="Bruce Wayne"
)

Hero(id=None, name='Batman', secret_name='Bruce Wayne', age=None, team_id=None)

In [5]:
sqlite_file_name = "tmp/tutorial.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

In [6]:
engine = create_engine(sqlite_url, echo=True)

In [7]:
engine

Engine(sqlite:///tmp/tutorial.db)

In [8]:
SQLModel.metadata.create_all(engine)

2022-01-09 15:09:57,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:57,766 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2022-01-09 15:09:57,766 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-09 15:09:57,767 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2022-01-09 15:09:57,768 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-09 15:09:57,769 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("team")
2022-01-09 15:09:57,769 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-09 15:09:57,770 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("team")
2022-01-09 15:09:57,770 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-09 15:09:57,771 INFO sqlalchemy.engine.Engine 
CREATE TABLE team (
	id INTEGER, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2022-01-09 15:09:57,771 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2022-01-09 15:09:57,773 INFO sqlalchemy.engine.Engine CREATE INDEX ix_team_name ON team (name)
2022-01-09 15:0

In [9]:
def create_heroes():
    with Session(engine) as session:
        # create teams
        team_preventers = Team(
            name="Preventers"
        )
        team_zmen = Team(
            name="Z-Men"
        )
        session.add(team_preventers)        
        session.add(team_zmen)
        # create heroes
        hero_1 = Hero(
            name="Batman",
            secret_name="Bruce Wayne",
            team=team_preventers,
        )
        hero_2 = Hero(
            name="Superman",
            secret_name="Clark Kent",
            team=team_preventers
        )
        hero_3 = Hero(
            name="Spiderman",
            secret_name="Peter Parker",
            team=team_zmen
        )
        session.add(hero_1)
        session.add(hero_2)
        session.add(hero_3)
        session.commit()



In [10]:
create_heroes()

2022-01-09 15:09:57,888 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:57,892 INFO sqlalchemy.engine.Engine INSERT INTO team (name) VALUES (?)
2022-01-09 15:09:57,895 INFO sqlalchemy.engine.Engine [generated in 0.00281s] ('Preventers',)
2022-01-09 15:09:57,897 INFO sqlalchemy.engine.Engine INSERT INTO team (name) VALUES (?)
2022-01-09 15:09:57,898 INFO sqlalchemy.engine.Engine [cached since 0.006222s ago] ('Z-Men',)
2022-01-09 15:09:57,900 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2022-01-09 15:09:57,900 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('Batman', 'Bruce Wayne', None, 1)
2022-01-09 15:09:57,902 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2022-01-09 15:09:57,902 INFO sqlalchemy.engine.Engine [cached since 0.00244s ago] ('Superman', 'Clark Kent', None, 1)
2022-01-09 15:09:57,902 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secr

## Read Data

In [11]:
def select_heroes():
    with Session(engine) as session:
        result = session.exec(
            select(Hero)
        )
        return result.all()

In [12]:
select_heroes()

2022-01-09 15:09:57,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:57,968 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero
2022-01-09 15:09:57,968 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2022-01-09 15:09:57,969 INFO sqlalchemy.engine.Engine ROLLBACK


/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


[Hero(secret_name='Bruce Wayne', team_id=1, age=None, name='Batman', id=1),
 Hero(secret_name='Clark Kent', team_id=1, age=None, name='Superman', id=2),
 Hero(secret_name='Peter Parker', team_id=2, age=None, name='Spiderman', id=3)]

In [13]:
def select_by_name(name: str):
    with Session(engine) as session:
        results = session.exec(
            (
                select(Hero)
                .where(Hero.name == name)
            )

        )
        return results.one()

In [14]:
select_by_name("Batman")

2022-01-09 15:09:58,025 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,027 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2022-01-09 15:09:58,028 INFO sqlalchemy.engine.Engine [no key 0.00117s] ('Batman',)
2022-01-09 15:09:58,030 INFO sqlalchemy.engine.Engine ROLLBACK


Hero(secret_name='Bruce Wayne', team_id=1, age=None, name='Batman', id=1)

## Update

In [15]:
def update_age(
    hero_name: str,
    age: int
):
    with Session(engine) as session:
        hero = session.exec(
            (
                select(Hero)
                .where(Hero.name == hero_name)
            )
        ).one()
        hero.age = age
        session.add(hero)
        session.commit()
        session.refresh(hero)
        return hero

In [16]:
update_age("Batman", 42)

2022-01-09 15:09:58,087 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,089 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2022-01-09 15:09:58,091 INFO sqlalchemy.engine.Engine [no key 0.00212s] ('Batman',)
2022-01-09 15:09:58,093 INFO sqlalchemy.engine.Engine UPDATE hero SET age=? WHERE hero.id = ?
2022-01-09 15:09:58,093 INFO sqlalchemy.engine.Engine [generated in 0.00037s] (42, 1)
2022-01-09 15:09:58,094 INFO sqlalchemy.engine.Engine COMMIT
2022-01-09 15:09:58,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,096 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.id = ?
2022-01-09 15:09:58,097 INFO sqlalchemy.engine.Engine [generated in 0.00085s] (1,)
2022-01-09 15:09:58,098 INFO sqlalchemy.engine.Engine ROLLBACK


Hero(secret_name='Bruce Wayne', team_id=1, age=42, name='Batman', id=1)

## Delete

In [17]:
def delete_hero(
    name: str
):
    with Session(engine) as session:
        hero = session.exec(
            select(Hero)
            .where(Hero.name == name)
        ).one()
        print(hero)
        session.delete(hero)
        session.commit()
        

In [18]:
delete_hero("Superman")

2022-01-09 15:09:58,138 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,138 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id 
FROM hero 
WHERE hero.name = ?
2022-01-09 15:09:58,139 INFO sqlalchemy.engine.Engine [no key 0.00049s] ('Superman',)
secret_name='Clark Kent' team_id=1 age=None name='Superman' id=2
2022-01-09 15:09:58,140 INFO sqlalchemy.engine.Engine DELETE FROM hero WHERE hero.id = ?
2022-01-09 15:09:58,140 INFO sqlalchemy.engine.Engine [generated in 0.00026s] (2,)
2022-01-09 15:09:58,141 INFO sqlalchemy.engine.Engine COMMIT


## Select Connected Data

In [19]:
with Session(engine) as session:
    result = session.exec(
        (
            select(Hero, Team)
            .where(Hero.team_id == Team.id)
        )
    ).all()
result

2022-01-09 15:09:58,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,165 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1 
FROM hero, team 
WHERE hero.team_id = team.id
2022-01-09 15:09:58,165 INFO sqlalchemy.engine.Engine [no key 0.00030s] ()
2022-01-09 15:09:58,165 INFO sqlalchemy.engine.Engine ROLLBACK


/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class Select will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


[(Hero(secret_name='Bruce Wayne', team_id=1, age=42, name='Batman', id=1), Team(name='Preventers', id=1)),
 (Hero(secret_name='Peter Parker', team_id=2, age=None, name='Spiderman', id=3), Team(name='Z-Men', id=2))]

In [20]:
def join_heroes_and_teams():
    with Session(engine) as session:
        result = session.exec(
            select(Hero, Team)
            .join(Team)
        ).all()
        return result

In [21]:
join_heroes_and_teams()

2022-01-09 15:09:58,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:09:58,203 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1 
FROM hero JOIN team ON team.id = hero.team_id
2022-01-09 15:09:58,204 INFO sqlalchemy.engine.Engine [no key 0.00035s] ()
2022-01-09 15:09:58,204 INFO sqlalchemy.engine.Engine ROLLBACK


[(Hero(secret_name='Bruce Wayne', team_id=1, age=42, name='Batman', id=1), Team(name='Preventers', id=1)),
 (Hero(secret_name='Peter Parker', team_id=2, age=None, name='Spiderman', id=3), Team(name='Z-Men', id=2))]

## Pandas Test

In [22]:
import pandas

In [23]:
statement = select(Hero)

In [24]:
session = Session(engine)
pandas.read_sql(
    statement,
    con=sqlite_url
)

,id,name,secret_name,age,team_id
0,1,Batman,Bruce Wayne,42.0,1
1,3,Spiderman,Peter Parker,NaN,2


In [25]:
pandas.read_sql(
    (
        select(Hero, Team)
        .join(Team)
    ),
    con=sqlite_url
)

,id,name,secret_name,age,team_id,id_1,name_1
0,1,Batman,Bruce Wayne,42.0,1,1,Preventers
1,3,Spiderman,Peter Parker,NaN,2,2,Z-Men


## Relationship Attributes

In [26]:
def create_team_with_heroes():
    """Create a team with heroes."""
    with Session(engine) as session:
        black_panther = Hero(
            name="Black Panther",
            secret_name="T'Challa",
        )
        team_wakanda = Team(
            name="Wakanda",
            heroes=[black_panther]
        )
        session.add(team_wakanda)
        session.commit()


In [27]:
create_team_with_heroes()

2022-01-09 15:17:34,385 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-09 15:17:34,386 INFO sqlalchemy.engine.Engine INSERT INTO team (name) VALUES (?)
2022-01-09 15:17:34,386 INFO sqlalchemy.engine.Engine [cached since 456.5s ago] ('Wakanda',)
2022-01-09 15:17:34,387 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age, team_id) VALUES (?, ?, ?, ?)
2022-01-09 15:17:34,388 INFO sqlalchemy.engine.Engine [cached since 456.5s ago] ('Black Panther', "T'Challa", None, 3)
2022-01-09 15:17:34,389 INFO sqlalchemy.engine.Engine COMMIT
